# 1. Importing Libraries

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
#!pip install --upgrade "watson-developer-cloud>=2.5.1"
from watson_developer_cloud import PersonalityInsightsV3

import json
from os.path import join

import pandas as pd
from pandas.io.json import json_normalize

import numpy as np

# 2. PDF to TXT

In [3]:
#!pip install pdfminer.six
#import pdfminer
#pdf2txt.py ycombinator-sam-altman.pdf > ycombinator-sam-altman.txt

# 3. Data Files

In [4]:
#!git clone https://github.com/ssuleyma/InsideOut.git
path ='./InsideOut/Transcripts/'
files = os.listdir(path=path)
print(files)

['instagram-kevin-systrom.txt', 'facebook-mark-zuckerberg.txt', 'spotify-daniel-ek.txt', 'twitter-ev-williams.txt']


# 4. Watson Personality Insights (WPI) Authentication 

In [5]:
pi_version = "2017-10-13"
pi_api_key = ""
pi_url = "https://gateway.watsonplatform.net/personality-insights/api"

personality_insights = PersonalityInsightsV3(
    version = pi_version,
    iam_apikey = pi_api_key,
    url = pi_url
)

personality_insights.set_detailed_response(True)
personality_insights.set_default_headers({'x-watson-learning-opt-out': "true"})

# 5. Analysis by WPI

In [6]:
candidates_df = pd.DataFrame()
for i,f in enumerate(files):
    with open(join(path, f)) as profile_txt:
        profile = personality_insights.profile(profile_txt.read(),
                                               content_type='text/plain',
                                               accept='application/json',
                                               consumption_preferences=True,
                                               raw_scores=True).get_result()
        
        candidates_df = candidates_df.append(json_normalize(profile),ignore_index=True,sort=False)
        candidates_df.loc[i,'file'] = f

In [7]:
candidates_df.drop(columns=['processed_language','warnings'],inplace=True)

In [8]:
candidates_df.head()

,consumption_preferences,needs,personality,values,word_count,file
0,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2732,instagram-kevin-systrom.txt
1,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2517,facebook-mark-zuckerberg.txt
2,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",1854,spotify-daniel-ek.txt
3,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2852,twitter-ev-williams.txt


In [10]:
project.save_data('candidates_df.csv',candidates_df.to_csv(index=False),overwrite=True)

{'asset_id': '36131aed-e240-4df3-a8e8-5d9963aafb07',
 'bucket_name': 'insideout-donotdelete-pr-arlsgjghrogjm7',
 'file_name': 'candidates_df.csv',
 'message': 'File candidates_df.csv has been written successfully to the associated OS'}

# 6. Data Cleaning

In [5]:
# The code was removed by Watson Studio for sharing.

,consumption_preferences,needs,personality,values,word_count,file
0,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2732,instagram-kevin-systrom.txt
1,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2517,facebook-mark-zuckerberg.txt
2,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",1854,spotify-daniel-ek.txt
3,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2852,twitter-ev-williams.txt


In [6]:
def insights_to_df(insight, candidates):
    """
    Returns a dataframe of needs, values or personality raw scores.
    
    insight - "needs", "values", "personality"
    candidates - dataframe of candidates
    """
    n_v_df = pd.DataFrame()
    for r in candidates.index:
        d = pd.DataFrame(eval(candidates.copy().loc[r,insight]))[['name','raw_score']]
        d = d.set_index('name').T.reset_index(drop=True)
        d['file'] = candidates.loc[r,'file']
        n_v_df = n_v_df.append(d,ignore_index=True)
    n_v_df.columns.name = ""
    return n_v_df

def consumption(candidates):
    """
    Returns a dataframe of positive consumption preferences of candidates.
    
    candidates - dataframe of candidates
    """
    c_df = pd.DataFrame()
    for r in candidates.index:
        d = pd.DataFrame(eval(candidates.copy().loc[r,'consumption_preferences'])).drop(columns=['consumption_preference_category_id'])
        d['consumption_preferences'] = d['consumption_preferences'].apply(lambda l: [p['consumption_preference_id'][24:] for p in l if p['score'] >= 0.5])
        d = d[~d['name'].isin(['Music Preferences','Purchasing Preferences'])]
        d = d.set_index('name').T.reset_index(drop=True)
        d['file'] = candidates.loc[r,'file']
        c_df = c_df.append(d,ignore_index=True)
    c_df.columns.name = ""
    return c_df

def personality_details(candidates):
    """
    Returns dataframes of raw scores for each of the Big 5 traits.
    
    candidates - dataframe of candidates
    """
    open_df = pd.DataFrame()
    cons_df = pd.DataFrame()
    extra_df = pd.DataFrame()
    agree_df = pd.DataFrame()
    emotion_df = pd.DataFrame()
    
    for r in candidates.index:
        for t in range(0,5):
            t_name = eval(candidates_df.loc[r,'personality'])[t]['name']
            d = pd.DataFrame(eval(candidates_df.loc[r,'personality'])[t]['children'])[['name','raw_score']]
            d = d.set_index('name').T.reset_index(drop=True)
            d['file'] = candidates.loc[r,'file']
            
            if t_name == 'Openness':
                open_df = open_df.append(d,ignore_index=True)
            elif t_name == 'Conscientiousness':
                cons_df = cons_df.append(d,ignore_index=True)
            elif t_name == 'Extraversion':
                extra_df = extra_df.append(d,ignore_index=True)
            elif t_name == 'Agreeableness':
                agree_df = agree_df.append(d,ignore_index=True)
            elif t_name == 'Emotional range':
                emotion_df = emotion_df.append(d,ignore_index=True)
                
    open_df.columns.name = ""
    cons_df.columns.name = ""
    extra_df.columns.name = ""
    agree_df.columns.name = ""
    emotion_df.columns.name = ""
    
    return  open_df,cons_df,extra_df,agree_df,emotion_df

In [7]:
needs_df = insights_to_df('needs',candidates_df)
values_df = insights_to_df('values',candidates_df)
consumption_df = consumption(candidates_df)
all_personality_df = insights_to_df('personality',candidates_df)
openness_df, conscientiousness_df, extraversion_df, agreeableness_df, emotional_range_df = personality_details(candidates_df)

# 7. Exploratory Data Analysis

In [8]:
needs_df

,Challenge,Closeness,Curiosity,Excitement,Harmony,Ideal,Liberty,Love,Practicality,Self-expression,Stability,Structure,file
0,0.687388,0.737678,0.827021,0.553379,0.758527,0.643312,0.694281,0.691013,0.697178,0.646145,0.707389,0.694753,instagram-kevin-systrom.txt
1,0.729555,0.705474,0.834953,0.575228,0.740460,0.643764,0.698083,0.672824,0.700580,0.613063,0.695070,0.714984,facebook-mark-zuckerberg.txt
2,0.705916,0.734802,0.833490,0.580611,0.765428,0.656928,0.703968,0.696026,0.705324,0.623352,0.701928,0.709038,spotify-daniel-ek.txt
3,0.709902,0.723402,0.843319,0.544000,0.748655,0.668819,0.695866,0.696311,0.699126,0.627310,0.690413,0.701614,twitter-ev-williams.txt


In [9]:
values_df

,Conservation,Openness to change,Hedonism,Self-enhancement,Self-transcendence,file
0,0.575899,0.766610,0.633714,0.652945,0.829168,instagram-kevin-systrom.txt
1,0.598660,0.782782,0.609476,0.656638,0.829649,facebook-mark-zuckerberg.txt
2,0.586833,0.771245,0.639145,0.661032,0.830876,spotify-daniel-ek.txt
3,0.556791,0.775472,0.623224,0.655128,0.826246,twitter-ev-williams.txt


In [10]:
consumption_df

,Health & Activity Preferences,Environmental Concern Preferences,Entrepreneurship Preferences,Movie Preferences,Reading Preferences,Volunteering Preferences,file
0,[outdoor],[concerned_environment],[start_business],"[movie_adventure, movie_musical, movie_histori...","[read_frequency, books_non_fiction, books_auto...",[volunteer],instagram-kevin-systrom.txt
1,[outdoor],[concerned_environment],[start_business],"[movie_adventure, movie_historical, movie_scie...","[read_frequency, books_non_fiction, books_fina...",[volunteer],facebook-mark-zuckerberg.txt
2,[outdoor],[concerned_environment],[],"[movie_adventure, movie_musical, movie_histori...","[read_frequency, books_non_fiction, books_auto...",[volunteer],spotify-daniel-ek.txt
3,[outdoor],[concerned_environment],[start_business],"[movie_adventure, movie_historical, movie_scie...","[read_frequency, books_non_fiction, books_fina...",[volunteer],twitter-ev-williams.txt


In [11]:
all_personality_df

,Openness,Conscientiousness,Extraversion,Agreeableness,Emotional range,file
0,0.800396,0.627016,0.523315,0.721635,0.497048,instagram-kevin-systrom.txt
1,0.795236,0.660267,0.512500,0.697280,0.497587,facebook-mark-zuckerberg.txt
2,0.791975,0.574534,0.455835,0.704284,0.473716,spotify-daniel-ek.txt
3,0.818073,0.633957,0.528237,0.721584,0.506147,twitter-ev-williams.txt


In [12]:
openness_df

,Adventurousness,Artistic interests,Emotionality,Imagination,Intellect,Authority-challenging,file
0,0.519717,0.727920,0.669370,0.709328,0.712264,0.628538,instagram-kevin-systrom.txt
1,0.541532,0.691450,0.643814,0.691068,0.722693,0.623718,facebook-mark-zuckerberg.txt
2,0.529975,0.716827,0.678005,0.730441,0.736404,0.647658,spotify-daniel-ek.txt
3,0.536035,0.733576,0.670893,0.746914,0.741595,0.667499,twitter-ev-williams.txt


In [13]:
conscientiousness_df

,Achievement striving,Cautiousness,Dutifulness,Orderliness,Self-discipline,Self-efficacy,file
0,0.675763,0.615348,0.680129,0.440515,0.530816,0.697531,instagram-kevin-systrom.txt
1,0.728193,0.613682,0.681686,0.449990,0.574116,0.761576,facebook-mark-zuckerberg.txt
2,0.686923,0.588714,0.682981,0.429911,0.521362,0.723693,spotify-daniel-ek.txt
3,0.699108,0.590725,0.683971,0.436252,0.530293,0.722200,twitter-ev-williams.txt


In [14]:
extraversion_df

,Activity level,Assertiveness,Cheerfulness,Excitement-seeking,Outgoing,Gregariousness,file
0,0.534139,0.633161,0.583899,0.517096,0.494154,0.375204,instagram-kevin-systrom.txt
1,0.616067,0.702512,0.587455,0.548125,0.557755,0.428105,facebook-mark-zuckerberg.txt
2,0.538444,0.631072,0.571256,0.544202,0.492812,0.362288,spotify-daniel-ek.txt
3,0.570366,0.646632,0.580420,0.535335,0.514934,0.370557,twitter-ev-williams.txt


In [15]:
agreeableness_df

,Altruism,Cooperation,Modesty,Uncompromising,Sympathy,Trust,file
0,0.732964,0.655640,0.485216,0.697863,0.711541,0.598488,instagram-kevin-systrom.txt
1,0.743622,0.646228,0.450045,0.674064,0.719749,0.639157,facebook-mark-zuckerberg.txt
2,0.744824,0.665624,0.482940,0.705629,0.738680,0.613193,spotify-daniel-ek.txt
3,0.752197,0.669407,0.454430,0.683956,0.753108,0.631402,twitter-ev-williams.txt


In [16]:
emotional_range_df

,Fiery,Prone to worry,Melancholy,Immoderation,Self-consciousness,Susceptible to stress,file
0,0.437068,0.532793,0.481805,0.518504,0.576679,0.476039,instagram-kevin-systrom.txt
1,0.410326,0.532186,0.441051,0.498628,0.552278,0.429566,facebook-mark-zuckerberg.txt
2,0.422803,0.555647,0.493483,0.525057,0.608136,0.469477,spotify-daniel-ek.txt
3,0.444969,0.567656,0.477213,0.497748,0.595427,0.477262,twitter-ev-williams.txt


# 8. IBM 1-3-9

<img src="https://upload.wikimedia.org/wikipedia/commons/1/1d/IBM_values.png" alt="IBM 139" style="width: 700px; height: 300px;"/>